In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

from shapkit_nbdev.shapley_values import ShapleyValues
from shapkit_nbdev.inspector import inspector
from shapkit_nbdev.monte_carlo_shapley import MonteCarloShapley
from shapkit_nbdev.sgd_shapley import SGDshapley

%load_ext autoreload
%autoreload 2

# Load dataset

In [2]:
df = pd.read_csv("/home/sgrah/Documents/xAI/sophia_summit/compas_dataset/compas.csv")
df.age = df.age.astype("int")
df.priors_count = df.priors_count.astype("int")
df.columns = ['gender', 'ethnicity', 'legal_status', 'custody_status', 'marital_status',
              'recidivism_probability', 'age', 'priors_crimes_count']
df = df[df.recidivism_probability.isin(['Low', 'High'])]
df.head(5)

,gender,ethnicity,legal_status,custody_status,marital_status,recidivism_probability,age,priors_crimes_count
0,Male,Caucasian,Pretrial,Jail Inmate,Married,Low,31,0
1,Male,African-American,Pretrial,Jail Inmate,Single,High,21,1
3,Female,African-American,Pretrial,Jail Inmate,Single,Low,30,0
4,Male,Hispanic,Pretrial,Jail Inmate,Single,High,32,20
5,Female,Caucasian,Pretrial,Jail Inmate,Single,Low,49,0


In [3]:
target_names = 'recidivism_probability'
columns = [col for col in df.columns if col != target_names]
X = df[columns].copy()
labels = df[target_names].copy()
y = np.where(labels == 'Low', 0, 1)
print("Classes: {0}".format(np.unique(labels)))
X.head(3)

Classes: ['High' 'Low']


,gender,ethnicity,legal_status,custody_status,marital_status,age,priors_crimes_count
0,Male,Caucasian,Pretrial,Jail Inmate,Married,31,0
1,Male,African-American,Pretrial,Jail Inmate,Single,21,1
3,Female,African-American,Pretrial,Jail Inmate,Single,30,0



# Train a ML model

In [4]:
cat_features = ['gender', 'ethnicity', 'legal_status', 'custody_status', 'marital_status']
cat_features_index = [i for i, col in enumerate(X.columns) if col in cat_features]
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.33,
                                                    random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.33,
                                                  random_state=42)
test_data = catboost_pool = Pool(X_test, y_test,
                                 cat_features=cat_features_index)
val_data = catboost_pool = Pool(X_val, y_val,
                                cat_features=cat_features_index)

model = CatBoostClassifier(iterations=200,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=20)
# train the model
model.fit(X_train, y_train,
          cat_features=cat_features_index,
          eval_set=val_data)
# make the prediction using the resulting model
preds_class = model.predict(test_data)
preds_proba = model.predict_proba(test_data)
print("Test Accuracy: {0:.2}".format(np.mean(preds_class == y_test)))

0:	learn: 0.4642593	test: 0.4567796	best: 0.4567796 (0)	total: 59.5ms	remaining: 11.8s
20:	learn: 0.3708433	test: 0.3890843	best: 0.3890843 (20)	total: 234ms	remaining: 1.99s
40:	learn: 0.3393934	test: 0.3795197	best: 0.3793465 (36)	total: 373ms	remaining: 1.45s
60:	learn: 0.3238395	test: 0.3798016	best: 0.3765597 (54)	total: 511ms	remaining: 1.16s
80:	learn: 0.3088234	test: 0.3846077	best: 0.3765597 (54)	total: 664ms	remaining: 975ms
100:	learn: 0.3002132	test: 0.3852011	best: 0.3765597 (54)	total: 812ms	remaining: 796ms
120:	learn: 0.2900827	test: 0.3868944	best: 0.3765597 (54)	total: 959ms	remaining: 626ms
140:	learn: 0.2783800	test: 0.3916980	best: 0.3765597 (54)	total: 1.11s	remaining: 466ms
160:	learn: 0.2689697	test: 0.3942314	best: 0.3765597 (54)	total: 1.24s	remaining: 301ms
180:	learn: 0.2609107	test: 0.3952044	best: 0.3765597 (54)	total: 1.38s	remaining: 145ms
199:	learn: 0.2514048	test: 0.3948909	best: 0.3765597 (54)	total: 1.5s	remaining: 0us

bestTest = 0.3765596588
bestI

# Define the game

In [5]:
d = X_train.shape[1]
n = 2**d - 2
d, n

(7, 126)

In [6]:
fc = lambda x: model.predict(x)

In [7]:
r_class, x_class = 0, 0
while x_class == r_class:
    idx_r, idx_x = np.random.choice(np.arange(len(X_test)), size=2, replace=False)
    r = X_test.iloc[idx_r,:]
    x = X_test.iloc[idx_x,:]
    r_class = fc(r.values)
    x_class = fc(x.values)
fc_class = lambda x: 1 if int(fc(x)) == int(x_class) else 0

In [8]:
print(r)
print()
print("Class Prediction for r: {0:.0f}".format(model.predict(r.values)))
print("Real class for r: {0:.0f}".format(y_test[idx_r]))

gender                             Male
ethnicity              African-American
legal_status                   Pretrial
custody_status              Jail Inmate
marital_status                   Single
age                                  29
priors_crimes_count                   0
Name: 20669, dtype: object

Class Prediction for r: 0
Real class for r: 0


In [9]:
print(x)
print()
print("Class Prediction for x: {0:.0f}".format(model.predict(x.values)))
print("Real class for x: {0:.0f}".format(y_test[idx_x]))

gender                               Male
ethnicity                       Caucasian
legal_status                     Pretrial
custody_status         Pretrial Defendant
marital_status                     Single
age                                    50
priors_crimes_count                    16
Name: 3449, dtype: object

Class Prediction for x: 1
Real class for x: 1


# Exact Shapley Values

In [10]:
true_shap = ShapleyValues(x=x, fc=fc_class, r=r)

100%|██████████| 7/7 [00:00<00:00, 12.24it/s]


In [11]:
true_shap

gender                 0.000000
ethnicity             -0.083333
legal_status           0.000000
custody_status         0.250000
marital_status         0.000000
age                   -0.083333
priors_crimes_count    0.916667
dtype: float64

# Approximation methods

## Monte Carlo 

In [12]:
mc_shap = MonteCarloShapley(x=x, fc=fc_class, r=r, n_iter=100)
mc_shap

100%|██████████| 100/100 [00:00<00:00, 132.07it/s]


gender                 0.00
ethnicity             -0.05
legal_status           0.00
custody_status         0.31
marital_status         0.00
age                   -0.14
priors_crimes_count    0.88
dtype: float64

## SGD

In [13]:
sgd_est = SGDshapley(d, C=y.max())
sgd_shap = sgd_est.sgd(x=x, fc=fc_class, r=r, n_iter=1000, step=.1, step_type="sqrt")
sgd_shap

100%|██████████| 1000/1000 [00:01<00:00, 779.91it/s]


gender                 0.013803
ethnicity             -0.063308
legal_status           0.014338
custody_status         0.276489
marital_status         0.019682
age                   -0.081754
priors_crimes_count    0.820751
dtype: float64